## How many deploy notebooks are there?!
Boa version was intended to be used because of jupyter web wallet integrations.
However, for complex interactions, simulation times are not feasible, so we're rewriting all in web3py.

## Secret PK loading

In [ ]:
import os
from secure_key_utils import get_web3_account
from getpass import getpass

ENCRYPTED_PK = os.environ.get("ENCRYPTED_PK")  # encrypted blob
deployer = get_web3_account(ENCRYPTED_PK, getpass())  # decrypts with local pass + keychain

print(f"Deployer address: {deployer.address}")

# 1. Initialization

### i) Configure RPCs & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from web3 import Web3
import logging
import subprocess
import time

from ABIs import createX_abi

logging.basicConfig(level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")

load_dotenv()
main_chain = "ethereum"

DRPC_API_KEY = os.environ.get("DRPC_API_KEY")
ANKR_API_KEY = os.environ.get("ANKR_API_KEY")

RPCs = {
    # ankr everywhere (reliable for deployments, drpc as fallback)
    "ethereum": "https://rpc.ankr.com/eth/" + ANKR_API_KEY,
    "arbitrum": "https://rpc.ankr.com/arbitrum/" + ANKR_API_KEY,
    "polygon": "https://rpc.ankr.com/polygon/" + ANKR_API_KEY,
    "bsc": "https://rpc.ankr.com/bsc/" + ANKR_API_KEY,
    "base": "https://rpc.ankr.com/base/" + ANKR_API_KEY,
    "avalanche": "https://rpc.ankr.com/avalanche/" + ANKR_API_KEY,
    "optimism": "https://rpc.ankr.com/optimism/" + ANKR_API_KEY,
    "fraxtal": "https://lb.drpc.org/ogrpc?network=fraxtal&dkey=" + DRPC_API_KEY,
    "gnosis": "https://rpc.ankr.com/gnosis/" + ANKR_API_KEY,
    "sonic": "https://rpc.ankr.com/sonic_mainnet/" + ANKR_API_KEY,
    "ink": "https://lb.drpc.org/ogrpc?network=ink&dkey=" + DRPC_API_KEY,
    "mp1": "https://rpc.ankr.com/corn_maizenet",  # corn
    # "fantom": "https://lb.drpc.org/ogrpc?network=fantom&dkey="+DRPC_API_KEY, # Must redeploy later with another evm
    # "xlayer": "https://rpc.ankr.com/xlayer/"+ANKR_API_KEY,  # no mcopy
    # "taiko": "https://lb.drpc.org/ogrpc?network=taiko&dkey="+DRPC_API_KEY, # no push0
}

createX_address = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"

state_dict = {}


for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))
    state_dict[key]["createx"] = boa.loads_abi(createX_abi).at(createX_address)

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        logging.info(f"Working with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        logging.info(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.6f} ETH"
        )
        logging.info(f"CreateX test: {state_dict[key]['createx'].computeCreate2Address(b'',b'')}")

### ii) Parse LZ deployments data (libs and endpoints)

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
        }
    )

    # Print info
    logging.info(f"LZ details for {chain}:")
    logging.info(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    logging.info(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    logging.info(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n---"
    )

# 2. Contracts deployment

In [ ]:
# populate previously deployed addresses
state_dict["ethereum"]["oracle"] = "0xB10CFACE40490D798770FEdd104e0a013eD308a6"
state_dict["arbitrum"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["polygon"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["bsc"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["base"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["avalanche"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["optimism"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["fraxtal"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["gnosis"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["sonic"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["ink"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"
state_dict["mp1"]["oracle"] = "0xb10cfaceAF164682b8a9479B3A63A814BADa5ee0"

for key in state_dict:
    pre_deployed = state_dict[key].get("oracle", None)
    if hasattr(pre_deployed, "address"):
        pre_deployed = pre_deployed.address
    if pre_deployed:
        logging.info(f"Contract view deployed at {pre_deployed} on {key}")

### i) Deploy main view contract and block oracles 

In [ ]:
# On mainnet viewer will tell us ground truth block data
# On other chains block oracles are consensus of various messengers (LZ in this script)

salt_view = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e80058c81d87d8bfb3019a17cf")
salt_oracle = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e80004e5027427c96c004d2336")

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        pre_deployed = state_dict[key].get("oracle", None)

        if hasattr(pre_deployed, "address"):
            # already a boa contract
            continue

        if key == main_chain:
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            deploycode = contract_deployer.compiler_data.bytecode
            if pre_deployed is None:
                address = state_dict[key]["createx"].deployCreate2(salt_view, deploycode)
                logging.info(f"Block view deployed at {address} on {key}")
            else:
                address = pre_deployed
                logging.info(f"Pre-deployed at {address} on {key}")
        else:
            # deploying block oracle on other chains
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            bytecode = contract_deployer.compiler_data.bytecode
            args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
            deploycode = bytecode + args
            if pre_deployed is None:
                address = state_dict[key]["createx"].deployCreate2(salt_oracle, deploycode)
                time.sleep(1)
                logging.info(f"Block oracle deployed at {address} on {key}")
            else:
                address = pre_deployed
                logging.info(f"Pre-deployed at {address} on {key}")
        state_dict[key]["oracle"] = contract_deployer.at(address)

In [ ]:
# optional (at redeploys of messengers) - check committers! (and remove if needed)
for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            continue
        oracle = state_dict[key]["oracle"]
        committers = oracle.get_all_committers()
        logging.info(f"Committers on {key}: {committers}")

In [ ]:
# populate previously deployed addresses
state_dict["arbitrum"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["polygon"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["bsc"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["base"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["avalanche"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["optimism"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["fraxtal"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["gnosis"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["sonic"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"
state_dict["ink"]["block_relay"] = "0xfacefeedfA293d661221c09a7ee70ef40A73D25E"

for key in state_dict:
    pre_deployed = state_dict[key].get("block_relay", None)
    if hasattr(pre_deployed, "address"):
        pre_deployed = pre_deployed.address
    if pre_deployed:
        logging.info(f"Relayer deployed at {pre_deployed} on {key}")

In [ ]:
# Deploy LZBlockRelay on each chain
# This contract is used to send and receive messages via LZ, including LZRead and chained broadcasting
salt_relay = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e8004e9cd17bdd140300dc4be8")
for key in state_dict:
    logging.info(f"Switching to {key}...")
    if key == main_chain:
        continue

    pre_deployed = state_dict[key].get("block_relay", None)
    if hasattr(pre_deployed, "address"):
        # already a boa contract
        continue

    with boa.swap_env(state_dict[key]["boa"]):
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        bytecode = contract_deployer.compiler_data.bytecode
        args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
        deploycode = bytecode + args
        if pre_deployed is None:
            address = state_dict[key]["createx"].deployCreate2(salt_relay, deploycode)
            time.sleep(1)
            logging.info(f"Messenger deployed at {address} on {key}")
        else:
            address = pre_deployed
            logging.info(f"Pre-deployed at {address} on {key}")

        contract = contract_deployer.at(address)
        state_dict[key]["block_relay"] = contract

# 3. Contracts configuration

## i) Oracles configuration

#### Initialize with all deployed peer ids and read libs

In [ ]:
# Prepare initialization parameters for block relay contracts
for key in state_dict.keys():
    if key == main_chain:
        continue

    # Basic parameters
    endpoint = state_dict[key]["endpoint"]
    default_gas_limit = 300_000
    read_channel = 4294967295 if state_dict[key]["read_lib"] != "unavailable" else 0

    # Use dict to manage unique peer relationships
    peers_dict = {}

    # 1. If read-enabled:
    # - add self as read peer
    # - add all other chains as peers for broadcasting
    if state_dict[key]["read_lib"] != "unavailable":
        peers_dict[read_channel] = state_dict[key]["block_relay"].address

        for target_key in state_dict.keys():
            if target_key != main_chain and target_key != key:
                peers_dict[state_dict[target_key]["eid"]] = state_dict[target_key][
                    "block_relay"
                ].address

    # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
    # duplicates are managed by dict structure
    for source_key in state_dict.keys():
        if (
            source_key != main_chain
            and source_key != key
            and state_dict[source_key]["read_lib"] != "unavailable"
        ):
            peers_dict[state_dict[source_key]["eid"]] = state_dict[source_key][
                "block_relay"
            ].address

    # Convert dict to lists
    peer_eids = list(peers_dict.keys())
    peers = list(peers_dict.values())

    # Prepare lib configuration
    channels = []
    libs = []
    lib_types = []

    # Add send and receive libs
    if state_dict[key]["send_lib"] != "unavailable":
        channels.append(state_dict[key]["eid"])
        libs.append(state_dict[key]["send_lib"])
        lib_types.append(1)

    if state_dict[key]["receive_lib"] != "unavailable":
        channels.append(state_dict[key]["eid"])
        libs.append(state_dict[key]["receive_lib"])
        lib_types.append(2)

    # Add read libs if available
    if state_dict[key]["read_lib"] != "unavailable":
        channels.extend([read_channel, read_channel])
        libs.extend([state_dict[key]["read_lib"], state_dict[key]["read_lib"]])
        lib_types.extend([1, 2])

    logging.info(f"init parameters on {key}")
    logging.info(f"Peer_eids: {peer_eids}")
    logging.info(f"Peers: {peers}")
    logging.info(f"Channels: {channels}")
    logging.info(f"Libs: {libs}")
    logging.info(f"Lib types: {lib_types}")
    logging.info(f"Initializing block relay on {key}...")

    # Initialize
    with boa.swap_env(state_dict[key]["boa"]):
        relay_contract = state_dict[key]["block_relay"]
        if not relay_contract.is_initialized():
            relay_contract.initialize(
                endpoint,
                default_gas_limit,
                read_channel,
                peer_eids,
                peers,
                channels,
                libs,
                lib_types,
            )
        else:
            logging.info(f"Already initialized on {key}!")

#### Add block oracles and committer

In [ ]:
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        relay_contract = state_dict[key]["block_relay"]

        # add block oracles to every relayer
        if relay_contract.block_oracle() == state_dict[key]["oracle"].address:
            logging.info(f"Skipping {key} - already set")
        else:
            relay_contract.set_block_oracle(state_dict[key]["oracle"].address)
            logging.info(f"Set block oracle tx on {key}")

        # now add relayer as committer to block oracle
        oracle_contract = state_dict[key]["oracle"]
        if oracle_contract.is_committer(relay_contract.address):
            logging.info(f"Skipping {key} - already a committer")
        else:
            oracle_contract.add_committer(relay_contract.address, True)
            logging.info(f"Add committer tx on {key}")

        # add mainnet view contract on read chains (as read source)
        if state_dict[key]["read_lib"] != "unavailable":
            if relay_contract.mainnet_block_view() != state_dict[main_chain]["oracle"].address:
                relay_contract.set_read_config(
                    True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
                )
                logging.info(f"Set read config on {key}")
            else:
                logging.info(f"Skipping {key} - already set")

## ii) LZ Configuration - removed - all at init

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
from web3.middleware import ExtraDataToPOAMiddleware


def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    try:
        state_dict[key]["w3"].middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)
    except Exception:
        pass
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["block_relay"].address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["oracle"].address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0, gas=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    if gas > 0:
        tx["gas"] = gas
    else:
        tx["gas"] = int(w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]

    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [])
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 3 * fee)
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Now separate broadcast

In [ ]:
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [fee for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

# broadcast call
func = broadcaster_w3.functions.broadcast_latest_block(receive_eids, broadcast_fees)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, sum(broadcast_fees))
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

### Now reads with broadcast

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "optimism"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
keys = [key for key in state_dict.keys() if key not in [broadcaster, main_chain]]
receive_eids = []
for key in keys:
    eid = state_dict[key]["eid"]
    print(f"Checking fee for {key}")
    try:
        broadcaster_w3.functions.quote_broadcast_fees([eid]).call()
        receive_eids.append(eid)
    except Exception as e:
        print(f"Error: {e}")
print(list(zip(keys, receive_eids)))
all_peers = broadcaster_w3.functions.get_configured_eids().call()
print(all_peers)
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [int(fee * 1.1) for fee in broadcast_fees]
logging.info(
    f"LZSend fees: {broadcast_fees} on {broadcaster}. Total: {sum(broadcast_fees)/1e18} ETH"
)

BROADCAST_GAS = 2_000_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# magic broadcast call
main_block = state_dict[main_chain]["w3"].eth.block_number
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Request remote lzread

In [ ]:
# Pick a read-supporting chain to request the read from
read_chain = "arbitrum-sepolia"
requesting_chain = "optimism-sepolia"

# first use read contracts to estimate read and broadcast fees
fee_multiplier = 2
read_contract = state_dict[read_chain]["block_relay_w3"]
read_fee = fee_multiplier * read_contract.functions.quote_read_fee().call()
broadcast_fee = (
    fee_multiplier
    * read_contract.functions.quote_broadcast_fees([state_dict[requesting_chain]["eid"]]).call()[0]
)
logging.info(f"Read fee: {read_fee} on {read_chain}")
logging.info(f"Broadcast fee: {broadcast_fee} on {read_chain}")

# now request remote read
requesting_contract = state_dict[requesting_chain]["block_relay_w3"]
func = requesting_contract.functions.request_remote_read(
    state_dict[read_chain]["eid"], read_fee, broadcast_fee
)
msg_value = 2 * (read_fee + broadcast_fee)
tx_hash = send_tx(state_dict[requesting_chain]["w3"], func, account, msg_value)
logging.info(f"Tx: {tx_hash.hex()} on {requesting_chain}")

### DVN Setup

In [ ]:
# first set the delegate
# func = contract_w3.functions.set_lz_delegate(account.address)
# tx_hash = send_tx(func, account)
# logging.info(f"Added self as delegate: {tx_hash.hex()}")
def checksum(address):
    return Web3.to_checksum_address(address)


for key in state_dict.keys():
    if key == main_chain:
        continue
    print(f"Setting up {key}")
    CONFIG_TYPE_READ = 1
    CONFIG_TYPE_ULN = 2
    # list all dvns
    # print("Listing all dvns")
    dvns_all = state_dict[key]["dvns"]
    # for dvn in dvns_all:
    #     if 'layerzero' in dvn['id'] or 'nether' in dvn['id']:
    #         print(dvn)
    contract = state_dict[key]["block_relay"]
    contract_w3 = state_dict[key]["block_relay_w3"]
    SEND_LIB = checksum(state_dict[key]["send_lib"])
    RECEIVE_LIB = checksum(state_dict[key]["receive_lib"])
    READ_LIB = state_dict[key]["read_lib"]
    LZ_EID = state_dict[key]["eid"]

    oapp = checksum(contract.address)
    confirmations = 1
    lz_dvns = [dvn for dvn in dvns_all if dvn["id"] == "layerzero-labs"]
    nm_dvns = [dvn for dvn in dvns_all if dvn["id"] == "nethermind"]

    # if there is more than 1 lz or nm dvn, we filter by lzReadCompatible==false
    if len(lz_dvns) > 1:
        lz_dvns = [
            dvn
            for dvn in dvns_all
            if dvn["id"] == "layerzero-labs" and not dvn.get("lzReadCompatible", False)
        ]
    if len(nm_dvns) > 1:
        nm_dvns = [
            dvn
            for dvn in dvns_all
            if dvn["id"] == "nethermind" and not dvn.get("lzReadCompatible", False)
        ]

    lz_nm = lz_dvns + nm_dvns
    # print(optional_dvns_raw)
    required_dvns = [dvn["address"] for dvn in lz_nm]
    required_dvns = [dvn["address"] for dvn in lz_dvns]  # only lz dvns
    optional_dvns = []
    # # all other dvs are optional, with a threshold of 2 (or less is there is not enough)
    optional_dvn_threshold = 0  # 1 out of 2

    # # sort dvns alphabetically (important, otherwise lz will fail!)
    required_dvns.sort()
    optional_dvns.sort()
    required_dvns = [checksum(dvn) for dvn in required_dvns]
    optional_dvns = [checksum(dvn) for dvn in optional_dvns]

    print("DVNs for send/receive:")
    print(f"Required: {required_dvns}")
    print(f"Optional: {optional_dvns}")
    print(f"Optional threshold: {optional_dvn_threshold}")
    # if key in list(state_dict.keys())[:8]:
    #     continue
    if key == "optimism":
        continue
    # all configured peers
    eids_to_set = contract_w3.functions.get_configured_eids().call()
    # no self and no read channel
    libs_to_set = []
    if READ_LIB.lower() == "unavailable":
        # we are on non-read, so must only receive from broadcasters
        libs_to_set = [RECEIVE_LIB]
    else:
        # we can both broadcast and receive
        libs_to_set = [SEND_LIB, RECEIVE_LIB]
    eids_to_set = [eid for eid in eids_to_set if eid != LZ_EID and eid < 4294965694]
    print(f"Eids to set: {eids_to_set}")
    eids_to_set = [30111]  # only receive from optimism
    libs_to_set = [RECEIVE_LIB]
    for eid in eids_to_set:
        for lib in libs_to_set:
            func = contract_w3.functions.set_lz_uln_config(
                eid,
                oapp,
                lib,
                CONFIG_TYPE_ULN,
                confirmations,
                required_dvns,
                optional_dvns,
                optional_dvn_threshold,
            )
            tx_hash = send_tx(state_dict[key]["w3"], func, account)
            print(f"Added ULN config to {eid} on {key}: {tx_hash.hex()}")
            time.sleep(2)

    # # read
    # if READ_LIB.lower() == "unavailable":
    #     print("READ_LIB is unavailable")
    #     continue

    # lz_dvns = [dvn for dvn in dvns_all if dvn["id"] == "layerzero-labs"]
    # nm_dvns = [dvn for dvn in dvns_all if dvn["id"] == "nethermind"]

    # # if there is more than 1 lz or nm dvn, we filter by lzReadCompatible==false
    # if len(lz_dvns) > 1:
    #     lz_dvns = [dvn for dvn in dvns_all if dvn["id"] == "layerzero-labs" and dvn.get("lzReadCompatible", False)]
    # if len(nm_dvns) > 1:
    #     nm_dvns = [dvn for dvn in dvns_all if dvn["id"] == "nethermind" and dvn.get("lzReadCompatible", False)]

    # lz_nm = lz_dvns + nm_dvns
    # # print(lz_nm)
    # required_dvns = [dvn["address"] for dvn in lz_nm]
    # optional_dvns = []
    # optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1 or 0
    # required_dvns.sort()
    # optional_dvns.sort()
    # required_dvns = [checksum(dvn) for dvn in required_dvns]
    # optional_dvns = [checksum(dvn) for dvn in optional_dvns]

    # print("DVNs for read:")
    # print(f"Required: {required_dvns}")
    # print(f"Optional: {optional_dvns}")
    # print(f"Optional threshold: {optional_dvn_threshold}")

    # read_channel = contract_w3.functions.LZ_READ_CHANNEL().call()
    # print(f"Read channel: {read_channel}")
    # if key in list(state_dict.keys())[:7]:
    #     continue

    # func = contract_w3.functions.set_lz_uln_config(
    #     read_channel,
    #     oapp,
    #     checksum(READ_LIB),
    #     CONFIG_TYPE_READ,
    #     confirmations,
    #     required_dvns,
    #     optional_dvns,
    #     optional_dvn_threshold,
    # )
    # tx_hash = send_tx(state_dict[key]["w3"], func, account)
    # print(f"Added ULN-Read config: {tx_hash.hex()}")
    # time.sleep(2)

In [ ]:
list(state_dict.keys())[:7]

## Validate abi on scans